In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pprint import pprint
from dotenv import load_dotenv
import pathlib
# from tqdm import tqdm
from tqdm.autonotebook import tqdm
from pathlib import Path
from pprint import pprint
import hashlib
import random

import polars as pl
from glob import glob

from thefuzz import process

from openai import OpenAI
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings

from google import genai
from google.genai import types
from langchain_openai import ChatOpenAI

import PIL.Image
import pymupdf


root_dir = Path(os.getcwd()).parent.parent
sys.path.insert(0, str(root_dir))

pl.Config.set_fmt_str_lengths(300)
pl.Config.set_tbl_rows(100)
pl.Config.set_tbl_cols(20);

C:\Users\edgarmp\AppData\Local\Temp\ipykernel_26168\2045030555.py:10: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
from src.d01_data.data import (read_json, json_dump, pdf_to_images, save_text_to_file)
from src.d00_utils.utils import(remove_duplicates, campos_to_srt,
                                get_doc_id_data,
                                campos_text_dict, normalize_string,
                                extract_section_names, match_words,
                                remove_number_parenthesis_space,
                                clean_translation, filter_rectangles_by_area)



from src.d03_modeling.modeling import get_imputation_campos, get_imputation_seleccion, translate
load_dotenv('../../.env')
 
raw_path = root_dir / 'data' / '01_raw'
intermediate_path = root_dir / 'data' / '02_intermediate'
clean_path = root_dir / 'data' / '03_clean'
output_path = root_dir / 'data' / '04_model_output'

In [3]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_MODEL = os.getenv('OPENAI_MODEL')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
GEMINI_MODEL = os.getenv('GEMINI_MODEL')

client = genai.Client(api_key=GEMINI_API_KEY)

In [4]:
idiomas = ['ingles', 'frances', 'aleman', 'ucraniano', 'arabe', 'chino_mandarin']
formularios = [Path(p) for p in sorted(glob(str(intermediate_path / 'formularios' / 'español' /'*/')))]

In [5]:
for formulario in tqdm(formularios):
    pdfs = [Path(p) for p in sorted(glob(str(formulario / '*.pdf')))]
    codigo_formulario = pdfs[0].parent.name
    
    for idioma in idiomas:
        for pdf in pdfs[:-1]:
            print(f'Traduciendo el pdf {pdf.name} al idioma {idioma}')
            pdf_index = pdf.name.split('_')[1]
            doc = pymupdf.open(pdf)
            page = doc[0]
            all_text = page.get_text()
            
            page_texts = [b[4].strip() for b in page.get_text('blocks')]

            normalized_page_texts = []
            for p in page_texts:
                normalized_page_texts.extend(normalize_string(p))
            normalized_page_texts = [remove_number_parenthesis_space(p) for p in normalized_page_texts if len(p) > 2]
            
            traducciones = translate(client=client,
                                model=GEMINI_MODEL,
                                language=idioma,
                                palabras='\n'.join(random.sample(normalized_page_texts, len(normalized_page_texts))),
                                contexto=all_text)

            traducciones = {trad.palabra:clean_translation(trad.traduccion) for trad in traducciones.parsed.traducciones}
            
            traduccion_json = clean_path / 'formularios' / 'traducciones' / idioma / codigo_formulario /f'{codigo_formulario}_{pdf_index}_{idioma}'
            os.makedirs(traduccion_json.parent, exist_ok=True)
            json_dump(traducciones, traduccion_json)
            
            redactions = []

            for search_text in normalized_page_texts:
                
                hits = page.search_for(search_text)
                if len(hits) == 0:
                    print(f'NO HIT for text: {search_text}')
                
                if len(hits) > 1:
                    hits = filter_rectangles_by_area(rectangles=hits, min_area=30)
                
                for hit in hits[:1]:
                    translate_key = match_words(word=search_text,
                                                candidates=list(traducciones.keys()),
                                                score_cutoff=80)
                    
                    if translate_key:
                        traduccion = traducciones[translate_key]
                    else:
                        print(f'Texto: {search_text} sin traducción')
                        traduccion = search_text
                    # hit.x1 += 20

                    redactions.append((hit, ' '.join(traduccion.split('\n'))))
                    page.add_redact_annot(hit, fill=(1, 1, 1))
                page.apply_redactions() 

            for rec, trad in redactions:
                page.insert_htmlbox(rec, trad)
            
            base_name = '_'.join(pdf.stem.split('_')[:-1])
            output_file_path = intermediate_path / 'formularios' / idioma / codigo_formulario / f'{base_name}_{idioma}{pdf.suffix}'
            os.makedirs(output_file_path.parent, exist_ok=True)
            doc.subset_fonts()
            doc.ez_save(output_file_path)
            doc.close()
            

  0%|          | 0/2 [00:00<?, ?it/s]

Traduciendo el pdf EX-10_0_español.pdf al idioma ingles
Archivo guardado exitosamente como c:\Users\edgarmp\Desktop\Proyectos\ODIASEIA hackaton\data\03_clean\formularios\traducciones\ingles\EX-10\EX-10_0_ingles.json
Traduciendo el pdf EX-10_1_español.pdf al idioma ingles
Archivo guardado exitosamente como c:\Users\edgarmp\Desktop\Proyectos\ODIASEIA hackaton\data\03_clean\formularios\traducciones\ingles\EX-10\EX-10_1_ingles.json
Traduciendo el pdf EX-10_0_español.pdf al idioma frances
Archivo guardado exitosamente como c:\Users\edgarmp\Desktop\Proyectos\ODIASEIA hackaton\data\03_clean\formularios\traducciones\frances\EX-10\EX-10_0_frances.json
Traduciendo el pdf EX-10_1_español.pdf al idioma frances
Archivo guardado exitosamente como c:\Users\edgarmp\Desktop\Proyectos\ODIASEIA hackaton\data\03_clean\formularios\traducciones\frances\EX-10\EX-10_1_frances.json
Traduciendo el pdf EX-10_0_español.pdf al idioma aleman
Archivo guardado exitosamente como c:\Users\edgarmp\Desktop\Proyectos\ODIA

 50%|█████     | 1/2 [04:05<04:05, 245.53s/it]

Traduciendo el pdf EX-17_0_español.pdf al idioma ingles
Archivo guardado exitosamente como c:\Users\edgarmp\Desktop\Proyectos\ODIASEIA hackaton\data\03_clean\formularios\traducciones\ingles\EX-17\EX-17_0_ingles.json
Traduciendo el pdf EX-17_1_español.pdf al idioma ingles
Archivo guardado exitosamente como c:\Users\edgarmp\Desktop\Proyectos\ODIASEIA hackaton\data\03_clean\formularios\traducciones\ingles\EX-17\EX-17_1_ingles.json
Traduciendo el pdf EX-17_0_español.pdf al idioma frances
Archivo guardado exitosamente como c:\Users\edgarmp\Desktop\Proyectos\ODIASEIA hackaton\data\03_clean\formularios\traducciones\frances\EX-17\EX-17_0_frances.json
Traduciendo el pdf EX-17_1_español.pdf al idioma frances
Archivo guardado exitosamente como c:\Users\edgarmp\Desktop\Proyectos\ODIASEIA hackaton\data\03_clean\formularios\traducciones\frances\EX-17\EX-17_1_frances.json
Traduciendo el pdf EX-17_0_español.pdf al idioma aleman
Archivo guardado exitosamente como c:\Users\edgarmp\Desktop\Proyectos\ODIA

100%|██████████| 2/2 [08:55<00:00, 267.73s/it]


In [6]:
idiomas = [Path(p) for p in sorted(glob(str(intermediate_path / 'formularios/*')))]
codigo_formularios = [p.name for p in formularios]

for idioma in idiomas:
    for codigo in codigo_formularios:
        paginas_formularios = sorted([Path(p) for p in sorted(glob(str(idioma / codigo /'*.pdf')))])
        
        first_doc = pymupdf.open(paginas_formularios[0])
        
        if len(paginas_formularios) > 2:
            for doc in paginas_formularios[1:-1]:
                first_doc.insert_pdf(pymupdf.open(doc))
        elif len(paginas_formularios) == 2:
            first_doc.insert_pdf(pymupdf.open(paginas_formularios[1])) 
        
        
        filename = paginas_formularios[0].name.split('_')
        output_file_path = output_path / 'formularios' / idioma.name / f'{"_".join(filename[:1] + filename[2:])}'
        os.makedirs(output_file_path.parent, exist_ok=True)
        first_doc.save(output_file_path)
        

In [7]:
# campos_dict = campos_text_dict(doc_id_data['Campos a rellenar'])

In [8]:
# section_names = extract_section_names(doc_id_data['Secciones a rellenar del formulario'])

In [9]:
# for w in normalized_page_texts[26:27]:
#     insts = page.search_for('DISCAPACITADO DE NACIONALIDAD ESPAÑOLA, QUE OTORGA DERECHO DE ARRAIGO')
#     print(len(insts))
#     for inst in insts[0:]:
#         page.add_redact_annot(inst, fill=(1,0,0))

#     # Apply the redactions to the current page
#     page.apply_redactions()

#     # Save the modified document
# doc.save('redacted_document.pdf')

In [10]:
        # page.draw_rect(hit, color=None, fill=pymupdf.pdfcolor['white'])
        # page.insert_htmlbox(hit, ' '.join(traduccion.split('\n')))
        
        
        # page.add_redact_annot(hit, ' '.join(traduccion.split('\n')), fontname='helv', fontsize=25)